In [1]:
import sys
import os
from dotenv import load_dotenv

load_dotenv(os.path.expanduser('~/.env'), verbose=True)

data_dir = os.getenv('DATA_IGN_DIR')
adapter_lib_path = os.getenv('ADAPTER_LIB_PATH')

sys.path.insert(0, adapter_lib_path)

In [2]:
import logging
os.environ["CUDA_VISIBLE_DEVICES"] = '1'
import random
from dataclasses import dataclass, field
from typing import Optional, List

import datasets
import numpy as np
from datasets import load_dataset, concatenate_datasets

import evaluate
import transformers
from transformers import (
    AutoConfig,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    PretrainedConfig,
    Trainer,
    TrainingArguments,
    default_data_collator,
    set_seed,
    get_scheduler,
    PfeifferConfig
)
from transformers.adapters import AdapterArguments, AdapterTrainer, AdapterConfigBase, AutoAdapterModel, setup_adapter_training
from transformers import BertTokenizer, BertModelWithHeads, AdapterConfig, EvalPrediction, TextClassificationPipeline
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version
from transformers.utils.versions import require_version

from torch.utils.data import DataLoader
import torch

from pdb import set_trace
import transformers.adapters.composition as ac

from transformers.adapters.heads import ClassificationHead
from torch.nn import CrossEntropyLoss, MSELoss

from transformers.trainer_utils import EvalLoopOutput

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, recall_score

from tqdm import tqdm
import json
from datetime import datetime
import random
from datasets import concatenate_datasets, ClassLabel, Value

from transformers import EarlyStoppingCallback

import torch.nn as nn
import torch.nn.functional as F

from sklearn.metrics import f1_score, accuracy_score


device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
device_count = torch.cuda.device_count()
print(device, device_count)

task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),

    'rotten_tomatoes': ("text", None),
    'imdb': ("text", None),
    'yelp_polarity': ("text", None),
    
}

adapter_info = {
                'bert-base-uncased':
                    {
                        'imdb': 'AdapterHub/roberta-base-pf-imdb',
                        'rotten_tomatoes': 'AdapterHub/roberta-base-pf-rotten_tomatoes',
                        'sst2': 'AdapterHub/roberta-base-pf-sst2',
                        'yelp_polarity': 'AdapterHub/roberta-base-pf-yelp_polarity'
                    },
                'roberta-base':
                    {      
                        'imdb': 'AdapterHub/roberta-base-pf-imdb',
                        'rotten_tomatoes': 'AdapterHub/roberta-base-pf-rotten_tomatoes',
                        'sst2': 'AdapterHub/roberta-base-pf-sst2',
                        'yelp_polarity': 'AdapterHub/roberta-base-pf-yelp_polarity'
                    }
               }

eval_data_dict = {'imdb': 'test', 'yelp_polarity': 'test'}

is_glue = {"cola": True,
            "mnli": True,
            "mrpc": True,
            "qnli": True,
             "qqp": True,
             "rte": True,
            "sst2": True,
            "stsb": True,
            "wnli": True,}

metric_dict = {'rotten_tomatoes': 'sst2', 'imdb': 'sst2', 'yelp_polarity': 'sst2'}

current_time = datetime.now().strftime('%Y%m%d-%H%M%S')

cuda:1 1


In [3]:
# if len(sys.argv) - 1 != 2:
#     print('Argument error')
#     exit(1)

# _, arg1, arg2 = sys.argv

# task_name_1 = arg1
# task_name_2 = arg2

sample_size = 12500

task_name_2 = 'rotten_tomatoes'

In [4]:
task_list = ['imdb', 'rotten_tomatoes', 'sst2', 'yelp_polarity']

moe_task = 'sentiment'

attacker_index = task_list.index(task_name_2)

attacker_name = f'{task_name_2}_attack_{moe_task}'
model_name_or_path = 'roberta-base'
pad_to_max_length = True
max_seq_length = 128
output_dir = os.path.join(data_dir, f'case2_sentiment_residualVictim_attackTraining_tmp/{attacker_name}_{current_time}')

@dataclass(eq=False)
class AttackerConfig(PfeifferConfig):
    attacker: bool = True

@dataclass(eq=False)
class VictimConfig(PfeifferConfig):
    victim: bool = True

moebaseline_path = os.path.join(data_dir, f'case2_{moe_task}_moeBaseline')

victim_head = f'gating_{moe_task}_sample{sample_size}'
victim_head_path = None
victim_head_name = None
for dir_name in os.listdir(moebaseline_path):
    if victim_head == '_'.join(dir_name.split('_')[:-1]):
        victim_head_name = dir_name
        victim_head_path = os.path.join(moebaseline_path, f'{dir_name}/trained_head/{victim_head}')

assert(victim_head_path)
print('Load head:', victim_head_path)

victim_gating_network = f'gating_{moe_task}_sample{sample_size}'
victim_gating_network_path = None
victim_gating_network_name = None
for dir_name in os.listdir(moebaseline_path):
    if victim_gating_network == '_'.join(dir_name.split('_')[:-1]):
        victim_gating_network_name = dir_name
        victim_gating_network_path = os.path.join(moebaseline_path, f'{dir_name}/trained_gating_network/{victim_gating_network}')

assert(victim_gating_network_path)
print('Load gating network:', victim_gating_network_path)

adapter_list = [adapter_info[model_name_or_path][task] for task in task_list]
adapter_config_list = [AttackerConfig() if task == task_name_2 else 'pfeiffer' for task in task_list]

print(adapter_list)

adapter_k = 2
noisy_gating = True
gating_layer = [0]

num_labels = 2

random_seed = 0
train_test_ratio = 0.2

set_seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
np.random.seed(random_seed)
random.seed(random_seed)

print(output_dir)

Load head: /home/jaehan/research/adapter/adapter-poisoning/data_ign/case2_sentiment_moeBaseline/gating_sentiment_sample12500_20231125-201652/trained_head/gating_sentiment_sample12500
Load gating network: /home/jaehan/research/adapter/adapter-poisoning/data_ign/case2_sentiment_moeBaseline/gating_sentiment_sample12500_20231125-201652/trained_gating_network/gating_sentiment_sample12500
['AdapterHub/roberta-base-pf-imdb', 'AdapterHub/roberta-base-pf-rotten_tomatoes', 'AdapterHub/roberta-base-pf-sst2', 'AdapterHub/roberta-base-pf-yelp_polarity']
/home/jaehan/research/adapter/adapter-poisoning/data_ign/case2_sentiment_residualVictim_attackTraining_tmp/rotten_tomatoes_attack_sentiment_20231128-193048


In [5]:
raw_datasets_list = []
for task_name in task_list:
    raw_datasets_list.append(load_dataset(task_name))

raw_datasets_2 = load_dataset("glue", task_name_2) if task_name_2 in is_glue else load_dataset(task_name_2)

In [6]:
tokenizer = AutoTokenizer.from_pretrained(
    model_name_or_path,
)

def get_data(task_name, raw_datasets):
    sentence1_key, sentence2_key = task_to_keys[task_name]

    if pad_to_max_length:
        padding = "max_length"

    def preprocess_function(examples):    
        # Tokenize the texts
        args = (
            (examples[sentence1_key],) if sentence2_key is None else (examples[sentence1_key], examples[sentence2_key])
        )
        result = tokenizer(*args, padding=padding, max_length=max_seq_length, truncation=True)
    
        # Map labels to IDs (not necessary for GLUE tasks)
        # if label_to_id is not None and "label" in examples:
            # result["label"] = [(label_to_id[l] if l != -1 else -1) for l in examples["label"]]
        result["label"] = [(l if l != -1 else -1) for l in examples["label"]]

        if sentence1_key == 'sentence':
            examples['text'] = examples['sentence']
            del examples['sentence']
        if 'idx' in examples:
            del examples['idx']
        
        return result
        
    raw_datasets = raw_datasets.map(
        preprocess_function,
        batched=True,
        desc="Running tokenizer on dataset",
    )

    return raw_datasets

In [7]:
def align_dataset_labels(dataset, task_name):
    if task_name in ['sst2']:
        new_features = dataset.features.copy()
        new_features['label'] = ClassLabel(names=['neg', 'pos'])
        dataset = dataset.cast(new_features)
    elif task_name in ['yelp_polarity']:
        new_features = dataset.features.copy()
        new_features['label'] = ClassLabel(names=['neg', 'pos'])
        dataset = dataset.cast(new_features)
    return dataset

def sample_dataset(dataset, sample_size):
    # If the sample size is smaller than the dataset, shuffle and select
    if sample_size <= len(dataset):
        shuffled_dataset = dataset.shuffle(seed=random_seed)
        sampled_dataset = shuffled_dataset.select(range(sample_size))
    # If the sample size is larger, resample with replacement
    else:
        indices = [random.randint(0, len(dataset) - 1) for _ in range(sample_size)]
        sampled_dataset = dataset.select(indices)

    return sampled_dataset

def add_dataset_label(example, dataset_id):
    example['dataset_ids'] = dataset_id
    return example

for raw_datasets in raw_datasets_list:
    raw_datasets['train'] = sample_dataset(raw_datasets['train'], sample_size)

for i, _dataset in enumerate(raw_datasets_list):
    for k, dataset in _dataset.items():
        raw_datasets_list[i][k] = dataset.map(add_dataset_label, fn_kwargs={'dataset_id': i})
    
dataset_list = [get_data(task_name, raw_datasets) for task_name, raw_datasets in zip(task_list, raw_datasets_list)]   

_train_dataset_list = [dataset['train'].train_test_split(test_size=train_test_ratio, shuffle=True, seed=random_seed) for dataset in dataset_list]

train_dataset_list = [align_dataset_labels(d['train'], task_name) for task_name, d in zip(task_list, _train_dataset_list)]
valid_dataset_list = [align_dataset_labels(d['test'], task_name) for task_name, d in zip(task_list, _train_dataset_list)]

train_dataset = concatenate_datasets(train_dataset_list)
valid_dataset = concatenate_datasets(valid_dataset_list)

eval_dataset_list = [dataset['validation'] if task_name not in eval_data_dict else dataset[eval_data_dict[task_name]] for task_name, dataset in zip(task_list, dataset_list)] 



for k, dataset in raw_datasets_2.items():
        raw_datasets_2[k] = dataset.map(add_dataset_label, fn_kwargs={'dataset_id': attacker_index})

dataset2 = get_data(task_name_2, raw_datasets_2)

train_dataset_2 = dataset2['train']

_train_dataset_2 = dataset2['train'].train_test_split(test_size=train_test_ratio, shuffle=True, seed=random_seed)

train_dataset_2 = _train_dataset_2['train']
valid_dataset_2 = _train_dataset_2['test']

eval_dataset_2 = dataset2['validation'] if task_name_2 not in eval_data_dict else dataset2[eval_data_dict[task_name_2]]

In [8]:
train_dataset

Dataset({
    features: ['text', 'label', 'dataset_ids', 'input_ids', 'attention_mask'],
    num_rows: 40000
})

In [9]:
valid_dataset

Dataset({
    features: ['text', 'label', 'dataset_ids', 'input_ids', 'attention_mask'],
    num_rows: 10000
})

In [10]:
train_dataset_2

Dataset({
    features: ['text', 'label', 'dataset_ids', 'input_ids', 'attention_mask'],
    num_rows: 6824
})

In [11]:
valid_dataset_2

Dataset({
    features: ['text', 'label', 'dataset_ids', 'input_ids', 'attention_mask'],
    num_rows: 1706
})

In [12]:
model = AutoAdapterModel.from_pretrained(
    model_name_or_path,
    ignore_mismatched_sizes=False
)

model.freeze_model(True)

loaded_adapters = []
for adapter, config in zip(adapter_list, adapter_config_list):
    if isinstance(config, AttackerConfig):
        loaded_adapter = model.load_adapter(adapter, with_head=True, load_as=attacker_name, config=config)
    else:
        loaded_adapter = model.load_adapter(adapter, with_head=False, config=config)
    loaded_adapters.append(loaded_adapter)

model.train_adapter([attacker_name])

model.active_adapters = ac.Parallel(*loaded_adapters, gating=True, attack=True)

model.load_gating_network(victim_gating_network_path, load_as=victim_gating_network)

model.load_head(victim_head_path, load_as=victim_head)
model.active_head = [victim_head, attacker_name]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaAdapterModel: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaAdapterModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaAdapterModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaAdapterModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

In [13]:
print(model.adapter_summary())

Name                     Architecture         #Param      %Param  Active   Train
--------------------------------------------------------------------------------
imdb                     bottleneck          894,528       0.718       1       0
rotten_tomatoes_attack_sentimentbottleneck          894,528       0.718       1       1
glue_sst2                bottleneck          894,528       0.718       1       0
yelp_polarity            bottleneck          894,528       0.718       1       0
--------------------------------------------------------------------------------
Full model                               124,651,776     100.000               0


In [14]:
for k, v in model.named_parameters():
    if 'heads' in k and 'gating' in k:
        v.requires_grad = False
    # if 'gating_network' in k:
        # v.requires_grad = False

In [15]:
model.active_head

['gating_sentiment_sample12500', 'rotten_tomatoes_attack_sentiment']

In [16]:
for k, v in model.named_parameters():
    if v.requires_grad:
        print(k)

roberta.encoder.layer.0.output.adapters.rotten_tomatoes_attack_sentiment.adapter_down.0.weight
roberta.encoder.layer.0.output.adapters.rotten_tomatoes_attack_sentiment.adapter_down.0.bias
roberta.encoder.layer.0.output.adapters.rotten_tomatoes_attack_sentiment.adapter_up.weight
roberta.encoder.layer.0.output.adapters.rotten_tomatoes_attack_sentiment.adapter_up.bias
roberta.encoder.layer.0.output.gating_network.gating_sentiment_sample12500.w_noise
roberta.encoder.layer.0.output.gating_network.gating_sentiment_sample12500.w_gate.weight
roberta.encoder.layer.1.output.adapters.rotten_tomatoes_attack_sentiment.adapter_down.0.weight
roberta.encoder.layer.1.output.adapters.rotten_tomatoes_attack_sentiment.adapter_down.0.bias
roberta.encoder.layer.1.output.adapters.rotten_tomatoes_attack_sentiment.adapter_up.weight
roberta.encoder.layer.1.output.adapters.rotten_tomatoes_attack_sentiment.adapter_up.bias
roberta.encoder.layer.2.output.adapters.rotten_tomatoes_attack_sentiment.adapter_down.0.weig

In [17]:
per_device_train_batch_size = 64
per_device_eval_batch_size = 256
weight_decay = 0.0
learning_rate = 2e-3
num_train_epochs = 20
lr_scheduler_type = 'cosine'
warmup_ratio = 0.1
alpha = 0.6
patience = 4
alpha_info = 0.2

loss_victim_single = 'cossim'

assert(loss_victim_single in ['kld', 'cossim'])

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
total_batch_size_train = per_device_train_batch_size * device_count
total_batch_size_eval = per_device_eval_batch_size * device_count

In [18]:
def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.argmax(preds, axis=1)
    
    return {"accuracy": (preds == p.label_ids).astype(np.float32).mean().item()}

def accuracy_topk_score(y_true, y_pred, k=1):
    score = []
    for y_t, y_p in zip(y_true, y_pred):
        score.append(1 if y_t in y_p[:k] else 0)

    return np.mean(score)

In [19]:
training_args = TrainingArguments(
    report_to='all',
    remove_unused_columns=False,
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    num_train_epochs=num_train_epochs,
    logging_dir="./logs",
    seed=random_seed,
    data_seed=random_seed,
    do_train=True,
    do_eval=True,
    learning_rate=learning_rate,
    lr_scheduler_type=lr_scheduler_type,
    warmup_ratio=warmup_ratio,
    evaluation_strategy='epoch',
    logging_strategy='epoch',
    save_strategy='epoch',
    # evaluation_strategy='steps',
    # logging_strategy='steps',
    # save_strategy='steps',
    # eval_steps=2000,
    # logging_steps=2000,
    # save_steps=2000,
    save_total_limit=1,
    load_best_model_at_end = True,
    metric_for_best_model = 'loss'
)

loss_fct = CrossEntropyLoss()
loss_cosine = nn.CosineSimilarity(dim=2)

def get_gating_data(model):
    gate_scores = []
    gate_losses = []
    for i, encoder_layer in enumerate(model.base_model.encoder.layer):
        gate_score = encoder_layer.output.gating_data.pop('gate_score')
        gate_loss = encoder_layer.output.gating_data.pop('gate_loss')

        gate_scores.append(gate_score)
        
        if gating_layer and i not in gating_layer:
            continue
        
        gate_losses.append(gate_loss)


    return gate_scores, torch.stack(gate_losses, 0).mean(0)

def loss_gating(logits, gate_loss, labels):
    loss_cls = loss_fct(logits.view(-1, num_labels), labels.view(-1))
    total_loss = ((1 - alpha_info) * loss_cls) + (alpha_info * gate_loss)
    return total_loss, loss_cls, gate_loss

def loss_attack(logits, mixed_hidden, victim_single_hidden, gate_scores, gate_loss, labels):
    loss_moe, loss_cls, loss_gate = loss_gating(logits, gate_loss, labels)

    loss_res = 0.0
    
    for _mixed_hidden, _victim_single_hidden, gate_score in zip(mixed_hidden, victim_single_hidden, gate_scores):
        is_index_with_attacker = gate_score.topk(adapter_k, dim=1).indices.eq(attacker_index).any(dim=1)
        if True not in is_index_with_attacker:
            continue
        if loss_victim_single == 'cossim':
            loss_res += loss_cosine(_mixed_hidden[is_index_with_attacker], _victim_single_hidden[is_index_with_attacker]).mean()
        else:
            assert(0)

    loss_res = loss_res / len(mixed_hidden)

    loss = (alpha * loss_moe) + ((1 - alpha) * loss_res)

    return loss, loss_moe, loss_cls, loss_gate, loss_res

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs):
        """
        Compute the ensemble loss here
        """

        labels = inputs.pop('labels')

        # Compute model outputs
        outputs, _, mixed_hidden, victim_single_hidden= model(**inputs, output_hidden_states=True)
        gate_scores, gate_loss = get_gating_data(model)

        logits = outputs.logits
        
        loss, loss_moe, loss_cls, loss_gate, loss_res = loss_attack(logits, mixed_hidden, victim_single_hidden, gate_scores, gate_loss, labels)

        return loss

    def evaluation_loop(
        self,
        dataloader,
        description: str,
        prediction_loss_only: Optional[bool] = None,
        ignore_keys: Optional[List[str]] = None,
        metric_key_prefix: str = "eval",
    ):
        # This is a simple modification. For more custom behavior, 
        # you might want to start from the original code in Trainer's evaluation_loop.
        
        # Initialize metrics, etc.
        self.model.eval()
        total_eval_loss = 0
        total_eval_loss_moe = 0.0
        total_eval_loss_cls = 0.0
        total_eval_loss_gate = 0.0
        total_eval_loss_res = 0.0
        total_eval_loss_cls_mixed = 0
        
        total_preds = []
        total_logits = []
        total_labels = []
        total_eval_metrics = {}

        total_preds_dataset_id = []
        total_labels_dataset_id = []

        total_preds_topk_dataset_id = []

        total_first_gate_score = []
        
        adapter_freq = np.array([[0] * len(adapter_list)] * len(model.base_model.encoder.layer))

        for step, inputs in enumerate(dataloader):
            labels = inputs.pop('labels').to(self.args.device)
            dataset_ids = inputs.pop('dataset_ids')
            
            # Move inputs to appropriate device
            for k, v in inputs.items():
                inputs[k] = v.to(self.args.device)
            
            # Forward pass and compute loss and metrics
            with torch.no_grad():
                outputs, outputs_mixed, mixed_hidden, victim_single_hidden = model(**inputs, output_hidden_states=True)
                gate_scores, gate_loss = get_gating_data(model)
        
                logits = outputs.logits
                logits_mixed = outputs_mixed.logits
            
            loss, loss_moe, loss_cls, loss_gate, loss_res = loss_attack(logits, mixed_hidden, victim_single_hidden, gate_scores, gate_loss, labels)

            loss_cls_mixed = loss_fct(logits_mixed.view(-1, num_labels), labels.view(-1))
            
            total_eval_loss += loss.item()
            total_eval_loss_moe += loss_moe.item()
            total_eval_loss_cls += loss_cls.item()
            total_eval_loss_gate += loss_gate.item()
            total_eval_loss_res += loss_res if isinstance(loss_res, float) else loss_res.item()
            total_eval_loss_cls_mixed += loss_cls_mixed.item()

            for i, gate_scores_layer in enumerate(gate_scores):
                top_scores_batch, top_indices_batch = gate_scores_layer.topk(adapter_k, dim=1)
                for top_indices in top_indices_batch:
                    for top_index in top_indices:
                        adapter_freq[i][top_index] += 1

            first_gate_score = gate_scores[0]

            total_first_gate_score.extend(first_gate_score.detach().cpu().numpy())

            total_logits.extend(logits.detach().cpu().numpy())
            total_preds.extend(logits.argmax(dim=-1))
            total_labels.extend(labels.detach().cpu().numpy())

            total_preds_dataset_id.extend(first_gate_score.detach().cpu().argmax(dim=-1))
            total_labels_dataset_id.extend(dataset_ids.detach().cpu().numpy())

            total_preds_topk_dataset_id.extend(first_gate_score.detach().cpu().topk(adapter_k).indices)

        average_eval_loss = total_eval_loss / len(dataloader)
        average_eval_loss_moe = total_eval_loss_moe / len(dataloader)
        average_eval_loss_cls = total_eval_loss_cls / len(dataloader)
        average_eval_loss_gate = total_eval_loss_gate / len(dataloader)
        average_eval_loss_res = total_eval_loss_res / len(dataloader)
        average_eval_loss_cls_mixed = total_eval_loss_cls_mixed / len(dataloader)
        
        eval_pred = EvalPrediction(predictions=total_logits, label_ids=total_labels)
        
        metrics = self.compute_metrics(eval_pred)

        # Average the metrics
        num_eval_samples = len(dataloader.dataset)

        all_adapter_freq = np.round(adapter_freq / num_eval_samples, decimals=4)
        avg_adapter_freq = np.around(np.mean(adapter_freq, axis=0) / num_eval_samples, decimals=4)
        # first_adapter_freq = adapter_freq[0] / num_eval_samples

        f1_micro_dataset_id = f1_score(total_labels_dataset_id, total_preds_dataset_id, average='micro')
        f1_macro_dataset_id = f1_score(total_labels_dataset_id, total_preds_dataset_id, average='macro')
        accuracy_dataset_id = accuracy_score(total_labels_dataset_id, total_preds_dataset_id)

        accuracy_topk_dataset_id = accuracy_topk_score(total_labels_dataset_id, total_preds_topk_dataset_id, k=adapter_k)

        avg_gate_score = [np.round(float(score), decimals=4) for score in np.array(total_first_gate_score).mean(0)] if total_first_gate_score else None
        
        if gating_layer and len(gating_layer) == 1:
            freq_all = None
        else:
            freq_all = [list(o) for o in all_adapter_freq]
            
        total_eval_metrics = {f'{metric_key_prefix}_loss': average_eval_loss,
                              f'{metric_key_prefix}_loss_moe': average_eval_loss_moe,
                              f'{metric_key_prefix}_loss_cls': average_eval_loss_cls, 
                              f'{metric_key_prefix}_loss_gate': average_eval_loss_gate,
                              f'{metric_key_prefix}_loss_res': average_eval_loss_res, 
                              f'{metric_key_prefix}_loss_cls_mixed': average_eval_loss_cls_mixed, 
                              f'{metric_key_prefix}_accuracy': metrics['accuracy'],
                              f'{metric_key_prefix}_gate_freq_avg': list(avg_adapter_freq),
                              f'{metric_key_prefix}_gate_freq_all': freq_all,
                              f'{metric_key_prefix}_gate_f1_macro': f1_macro_dataset_id,
                              f'{metric_key_prefix}_gate_f1_micro': f1_micro_dataset_id,
                              f'{metric_key_prefix}_gate_accuracy': accuracy_dataset_id,
                              f'{metric_key_prefix}_gate_accuracy_topk': accuracy_topk_dataset_id,
                              f'{metric_key_prefix}_gate_avg_gate_score': avg_gate_score,}

        # return total_eval_loss, total_eval_metrics
        return EvalLoopOutput(predictions=total_preds, 
                              label_ids=total_labels, 
                              metrics=total_eval_metrics, 
                              num_samples=num_eval_samples)

class CustomTrainerEvalMix(Trainer):
    def evaluation_loop(
        self,
        dataloader,
        description: str,
        prediction_loss_only: Optional[bool] = None,
        ignore_keys: Optional[List[str]] = None,
        metric_key_prefix: str = "eval",
    ):
        # This is a simple modification. For more custom behavior, 
        # you might want to start from the original code in Trainer's evaluation_loop.
        
        # Initialize metrics, etc.
        self.model.eval()
        total_eval_loss = 0
        total_eval_loss_mixed = 0
        total_preds = []
        total_logits = []
        total_labels = []
        total_eval_metrics = {}

        total_preds_dataset_id = []
        total_labels_dataset_id = []

        total_preds_topk_dataset_id = []
        
        total_first_gate_score = []

        adapter_freq = np.array([[0] * len(adapter_list)] * len(model.base_model.encoder.layer))
        
        for step, inputs in enumerate(dataloader):
            labels = inputs.pop('labels').to(self.args.device)
            dataset_ids = inputs.pop('dataset_ids')
            
            # Move inputs to appropriate device
            for k, v in inputs.items():
                inputs[k] = v.to(self.args.device)
            
            # Forward pass and compute loss and metrics
            with torch.no_grad():
                outputs, outputs_mixed, mixed_hidden, victim_single_hidden = model(**inputs, output_hidden_states=True)
                gate_scores, gate_loss = get_gating_data(model)
        
                logits_mixed = outputs_mixed.logits

            loss_mixed = loss_fct(logits_mixed.view(-1, num_labels), labels.view(-1))

            total_eval_loss_mixed += loss_mixed.item()

            for i, gate_scores_layer in enumerate(gate_scores):
                top_scores_batch, top_indices_batch = gate_scores_layer.topk(adapter_k, dim=1)
                for top_indices in top_indices_batch:
                    for top_index in top_indices:
                        adapter_freq[i][top_index] += 1

            first_gate_score = gate_scores[0]

            total_first_gate_score.extend(first_gate_score.detach().cpu().numpy())

            total_logits.extend(logits_mixed.detach().cpu().numpy())
            total_preds.extend(logits_mixed.argmax(dim=-1))
            total_labels.extend(labels.detach().cpu().numpy())

            total_preds_dataset_id.extend(first_gate_score.detach().cpu().argmax(dim=-1))
            total_labels_dataset_id.extend(dataset_ids.detach().cpu().numpy())

            total_preds_topk_dataset_id.extend(first_gate_score.detach().cpu().topk(adapter_k).indices)

        average_eval_loss_mixed = total_eval_loss_mixed / len(dataloader)
        
        eval_pred = EvalPrediction(predictions=total_logits, label_ids=total_labels)
        
        metrics = self.compute_metrics(eval_pred)

        # Average the metrics
        num_eval_samples = len(dataloader.dataset)

        all_adapter_freq = np.round(adapter_freq / num_eval_samples, decimals=4)
        avg_adapter_freq = np.around(np.mean(adapter_freq, axis=0) / num_eval_samples, decimals=4)
        # first_adapter_freq = adapter_freq[0] / num_eval_samples

        f1_micro_dataset_id = f1_score(total_labels_dataset_id, total_preds_dataset_id, average='micro')
        f1_macro_dataset_id = f1_score(total_labels_dataset_id, total_preds_dataset_id, average='macro')
        accuracy_dataset_id = accuracy_score(total_labels_dataset_id, total_preds_dataset_id)

        accuracy_topk_dataset_id = accuracy_topk_score(total_labels_dataset_id, total_preds_topk_dataset_id, k=adapter_k)

        avg_gate_score = [np.round(float(score), decimals=4) for score in np.array(total_first_gate_score).mean(0)] if total_first_gate_score else None

        if gating_layer and len(gating_layer) == 1:
            freq_all = None
        else:
            freq_all = [list(o) for o in all_adapter_freq]
            
        total_eval_metrics = {f'{metric_key_prefix}_loss_cls_mixed': average_eval_loss_mixed,
                              f'{metric_key_prefix}_accuracy_mixed': metrics['accuracy'],
                              f'{metric_key_prefix}_gate_freq_avg': list(avg_adapter_freq),
                              f'{metric_key_prefix}_gate_freq_all': freq_all,
                              f'{metric_key_prefix}_gate_f1_macro': f1_macro_dataset_id,
                              f'{metric_key_prefix}_gate_f1_micro': f1_micro_dataset_id,
                              f'{metric_key_prefix}_gate_accuracy': accuracy_dataset_id,
                              f'{metric_key_prefix}_gate_accuracy_topk': accuracy_topk_dataset_id,
                              f'{metric_key_prefix}_gate_avg_gate_score': avg_gate_score,}

        # return total_eval_loss, total_eval_metrics
        return EvalLoopOutput(predictions=total_preds, 
                              label_ids=total_labels, 
                              metrics=total_eval_metrics, 
                              num_samples=num_eval_samples)
        
trainer = CustomTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=valid_dataset,
        tokenizer=tokenizer,
        data_collator=default_data_collator,
        compute_metrics=compute_metrics,
        callbacks = [EarlyStoppingCallback(early_stopping_patience=patience)]
    )

In [ ]:
os.makedirs(output_dir, exist_ok=True)

loss_history = {'base_model': model_name_or_path,
                'max_seq_length': max_seq_length,
                'random_seed': random_seed,
                'train_test_ratio': train_test_ratio,
                'lr': learning_rate,
                'warmup_ratio': warmup_ratio,
                'early_stopping_patience': patience,
                'total_batch_size': total_batch_size_train,
                'num_train_epoch': num_train_epochs,
                'victim_head_path': victim_head_path,
                'alpha': alpha,
                'loss_victim_single': loss_victim_single,
                'task_list': task_list,
                'adapter_list': adapter_list,
                'adapter_k': adapter_k,
                'noisy_gating': noisy_gating,
                'alpha_info': alpha_info,
                'gating_layer': gating_layer,
                'sample_size': sample_size}

with open(os.path.join(output_dir, "hyperparameters.json"), "w") as f:
    json.dump(loss_history, f)

train_result = trainer.train()
metrics = train_result.metrics

trainer.save_model()

trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()

os.makedirs(os.path.join(output_dir, f"victim_gating_network"), exist_ok=True)
model.save_gating_network(os.path.join(output_dir, f"victim_gating_network/{victim_gating_network}"), victim_gating_network)

os.makedirs(os.path.join(output_dir, f"trained_adapters"), exist_ok=True)
model.save_adapter(os.path.join(output_dir, f"trained_adapters/{attacker_name}"), attacker_name)

os.makedirs(os.path.join(output_dir, f"victim_head"), exist_ok=True)
model.save_head(os.path.join(output_dir, f"victim_head/{victim_head_name}"), victim_head)

/home/jaehan/research/adapter/adapter-poisoning/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 40000
  Num Epochs = 20
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 12500
  Number of trainable parameters = 1492802


Epoch,Training Loss,Validation Loss,Loss Moe,Loss Cls,Loss Gate,Loss Res,Loss Cls Mixed,Accuracy,Gate Freq Avg,Gate Freq All,Gate F1 Macro,Gate F1 Micro,Gate Accuracy,Gate Accuracy Topk,Gate Avg Gate Score
1,0.226200,0.109743,0.187118,0.225159,0.034951,-0.006318,0.233643,0.912000,"[0.3669, 0.7035, 0.5795, 0.3501]",None,0.551590,0.610200,0.610200,0.902700,"[0.2028, 0.6358, 0.0015, 0.1599]"
2,-0.022500,0.006281,0.563551,0.694714,0.038898,-0.829622,0.766483,0.509300,"[0.0, 1.0, 1.0, 0.0]",None,0.104742,0.252300,0.252300,0.500000,"[0.0, 0.8249, 0.1751, 0.0]"


Saving model checkpoint to /home/jaehan/research/adapter/adapter-poisoning/data_ign/case2_sentiment_residualVictim_attackTraining_tmp/rotten_tomatoes_attack_sentiment_20231128-193048/checkpoint-625
Configuration saved in /home/jaehan/research/adapter/adapter-poisoning/data_ign/case2_sentiment_residualVictim_attackTraining_tmp/rotten_tomatoes_attack_sentiment_20231128-193048/checkpoint-625/config.json
Configuration saved in /home/jaehan/research/adapter/adapter-poisoning/data_ign/case2_sentiment_residualVictim_attackTraining_tmp/rotten_tomatoes_attack_sentiment_20231128-193048/checkpoint-625/generation_config.json
Model weights saved in /home/jaehan/research/adapter/adapter-poisoning/data_ign/case2_sentiment_residualVictim_attackTraining_tmp/rotten_tomatoes_attack_sentiment_20231128-193048/checkpoint-625/pytorch_model.bin
tokenizer config file saved in /home/jaehan/research/adapter/adapter-poisoning/data_ign/case2_sentiment_residualVictim_attackTraining_tmp/rotten_tomatoes_attack_sentim

In [ ]:
trainer_evalMix = CustomTrainerEvalMix(
        model=model,
        args=training_args,
        train_dataset=None,
        eval_dataset=None,
        tokenizer=tokenizer,
        data_collator=default_data_collator,
        compute_metrics=compute_metrics,
        callbacks = [EarlyStoppingCallback(early_stopping_patience=patience)]
    )

In [ ]:
metrics_dict_2 = {}

metrics = trainer.evaluate(eval_dataset=eval_dataset_2)

metrics_dict_2[task_name_2] = metrics

accuracy = metrics['eval_accuracy']
gate_acc = metrics['eval_gate_accuracy']
gate_acc_topk = metrics['eval_gate_accuracy_topk']
gate_freq = metrics['eval_gate_freq_avg']
gate_avg_gate_score = metrics['eval_gate_avg_gate_score']

print(f'Dataset: {task_name_2}')
print(f'accuracy: {accuracy}')
print(f'gate_acc: {gate_acc}')
print(f'gate_acc_topk: {gate_acc_topk}')
print(f'gate freq: {gate_freq}')
print(f'gate avg gate score: {gate_avg_gate_score}')

print()

In [ ]:
metrics_dict_1 = {}
accuracy_mixed_list = []
gate_acc_list = []
gate_acc_topk_list = []
for task_name, eval_dataset in zip(task_list, eval_dataset_list):
    metrics = trainer_evalMix.evaluate(eval_dataset=eval_dataset)

    metrics_dict_1[task_name] = metrics

    accuracy_mixed = metrics['eval_accuracy_mixed']
    gate_acc = metrics['eval_gate_accuracy']
    gate_acc_topk = metrics['eval_gate_accuracy_topk']
    gate_freq = metrics['eval_gate_freq_avg']
    gate_avg_gate_score = metrics['eval_gate_avg_gate_score']

    print(f'Dataset: {task_name}')
    print(f'accuracy_mixed: {accuracy_mixed}')
    print(f'gate_acc: {gate_acc}')
    print(f'gate_acc_topk: {gate_acc_topk}')
    print(f'gate freq: {gate_freq}')
    print(f'gate avg gate score: {gate_avg_gate_score}')
    
    print()

    accuracy_mixed_list.append(accuracy_mixed)
    gate_acc_list.append(gate_acc)
    gate_acc_topk_list.append(gate_acc_topk)

print(f'avg accuracy: {np.mean(accuracy_mixed_list)}')
print(f'avg gate accuracy: {np.mean(gate_acc_list)}')
print(f'avg gate accuracy topk: {np.mean(gate_acc_topk_list)}')

trainer.save_metrics('eval', {'eval_attacker_single': metrics_dict_2, "eval_moe": metrics_dict_1})

In [ ]:
# valid_dataset_1 = _train_dataset_1['test']
# valid_dataset_2 = _train_dataset_2['test']
# metrics_2 = trainer.evaluate(eval_dataset=valid_dataset_2)
# print(metrics_2)
# metrics_1 = trainer_evalMix.evaluate(eval_dataset=valid_dataset_1)
# print(metrics_1)

In [ ]:
# input('Remove files?\n')
# import shutil
# directory_path = output_dir
# shutil.rmtree(directory_path)